In [11]:
import numpy as np
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, transpile
from qiskit_aer import AerSimulator
import random

noiseprob = float(input("Enter Noise Probability:"))

def intpackets(num, bitlen=8):
    fmt = f'{{:0{bitlen}b}}'
    binstr = fmt.format(num)
    if len(binstr) > bitlen:
        raise ValueError(f"Number {num} too large for {bitlen} bits.")
    return [binstr[i:i+2] for i in range(0, len(binstr), 2)]

def superdense(bitpair, noiselevel=0.0):
    q = QuantumRegister(2, 'q')
    c = ClassicalRegister(2, 'c')
    qc = QuantumCircuit(q, c)
    qc.h(0)
    qc.cx(0, 1)
    if bitpair == '11':
        qc.z(0)
        qc.x(0)
    elif bitpair == '10':
        qc.z(0)
    elif bitpair == '01':
        qc.x(0)
    if noiselevel > 0 and random.random() < noiselevel:
        qc.x(0)
    qc.cx(0, 1)
    qc.h(0)
    qc.measure(q, c)
    return qc

def transmit(packets):
    simulator = AerSimulator()
    recbits = ""
    print(f"--- TRANSMITTING {len(packets)} PACKETS ---")
    for i, packet in enumerate(packets):
        qc = superdense(packet, noiseprob)
        job = simulator.run(transpile(qc, simulator), shots=1, memory=True)
        rawmeas = job.result().get_memory()[0][::-1]
        recbits += rawmeas
        status = "OK" if rawmeas == packet else "ERROR"
        print(f"Packet {i}: Sent '{packet}' -> Recv '{rawmeas}' [{status}]")
    return recbits
    
numinp = int(input("Enter Input Integer:")) 
bitwidth = 12 
print(f"Input: {numinp} (Noise: {noiseprob * 100}%)")
packets = intpackets(numinp, bitwidth)
print(f"Packets: {packets}")
streamout = transmit_data(packets)
output = int(streamout, 2)
    
print(f"Recieved Bin: {streamout}")
print(f"Output:   {output}")
    
if numinp == output:
    print("\nSUCCESS")
else:
    print(f"\nFAILURE (Diff: {numinp - output})")

Enter Noise Probability: 0.01
Enter Input Integer: 233


Input: 233 (Noise: 1.0%)
Packets: ['00', '00', '11', '10', '10', '01']
--- TRANSMITTING 6 PACKETS ---
Packet 0: Sent '00' -> Recv '00' [OK]
Packet 1: Sent '00' -> Recv '00' [OK]
Packet 2: Sent '11' -> Recv '11' [OK]
Packet 3: Sent '10' -> Recv '10' [OK]
Packet 4: Sent '10' -> Recv '10' [OK]
Packet 5: Sent '01' -> Recv '01' [OK]
Recv Bin: 000011101001
Output:   233

SUCCESS
